In [2]:
# Import Pyomo Modules

from __future__ import division
from pyomo.environ import *
import random
import numpy as np



In [3]:
# Define an empty AbstractModel() object
model = AbstractModel()

In [4]:
# Define the index sets for the grid, time horizon, and age class.

model.Iset = RangeSet(3) #e.g i = {1, 2, 3}
model.Jset = RangeSet(3)
model.Tset = RangeSet(7)
model.Kset = RangeSet(50)

In [11]:
# Defining all model parameters. 
# ALL PARAMETERS WILL MOST LIKELY NEED TO BE INDEXED AND SPECIFY A DEFAULT VALUE?

model.juvsurv = Param()   # Juvenille Tree Survivorship Probability
model.juvdeath = Param()  # Juvenille Tree Mortality Probability (model.juvdeath = 1 - model.juvsurv)
model.inventory = Param() # Tree inventory that is constant for each cell - acts as carrying capacity
model.beta = Param()      # Failure Rate of Infestation
model.R = Param()         # Growth rate of MPB per effective tree --> 1.8 per year, not indexed by time yet. 
model.c0 = Param()        # Fitting parameter for dispersal
model.c1 = Param()        # Fitting parameter for dispersal
model.gamma = Param()     # Inter-Resource Competition Parameter
model.lamb = Param()      # Effectiveness of Level 1 Control
model.rho = Param()       # Effectivneness of Level 2 Control


# Model Variables

### $$ model.juvenille -->  j_{k,t}^{(i,j)}$$ 

### $$ model.juvenilleTotal -->  J_{t}^{(i,j)} = \sum_{k=1}^{NJ}j_{k,t}^{(i,j)}, k = 1,2,...,NJ-1 $$

### $$ model.redsnag --> I_{t-1} $$

### $$ model.greysnag --> I_{t-2} $$

In [ ]:
# Defining all model variabeles:

model.juvenille = Var(model.Iset, model.Jset, model.Tset, model.Kset,          
                      within=NonNegativeReals, initialize='''FILL IN LATER''')

def juv_total(model, i, j, t, k):
    return sum(model.juvenille[k] for k in range(1,50))
model.juvenilleTotal = Var(model.Iset, model.Jset, model.Tset, model.Kset,
                           within=NonNegativeReals, initialize=juv_total)

model.susceptible = Var(model.Iset, model.Jset, model.Tset,
                       within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.infected = Var(model.Iset, model.Jset, model.Tset, model.Kset,
                    within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.redsnag = Var(model.Iset, model.Jset, model.Tset, model.Kset,
                   within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.greysnag = Var(model.Iset, model.Jset, model.Tset, model.Kset,
                    within=NonNegativeReals, initialize = '''FILL IN LATER''')

model.treeinventory = Var(model.Iset, model.Jset, model.Tset,
                        within=NonNegativeReals, initialize = '''FILL IN LATER w/ expr?''')


In [13]:
# Constraint 1: juvenilles that advance to the next age class (eq. 1)
def juv_advance(model, i, j, t, k):
    return model.juvenille[i,j,t+1,k+1] == model.juvenille[i,j,t,k]*model.juvsurv

# Constraint 2: total number of juvenilles in all age classes on cell (i,j)
def juv_total_sum(model, i, j, t, k):
    return """ This constraint is actually a variable declaration """

# Constraint 3: recruitment of seedlings to the first juvenille age class. 
def juv_recruit(model, i, j, t, k): """ Not sure how you would specify the I_t-2 var, will come back to this """
    return model.juvenille[i,j,t,1] == model.juvdeath*model.juvenilleTotal + model.greysnag
    
# Constraint 4: Infected growth rate - Leaving it as linear for now, will include exponential later when debugging.
def premigration_infectedgrowth(model, i, j, t):
    return model.infected[i,j,t+1] = model.R * model.infected[i,j,t]

# Constraint 5: Susceptible recruitment 
def susceptible_advance(model, i, j, t, k):
    return model.susceptible[i,j,t+1] == model.susceptible[i,j,t] - model.infected[i,j,t] \\
            + model.juvsurv*model.juvenille[i,j,t,k==50]

# Constraint 6: Tree inventory - constant for each cell. 
def tree_inventory(model, i, j, t):
    return model.inventory = model.juvenilleTotal[i,j,t+1] + model.susceptible[i,j,t+1] + model.infected[i,j,t] \\
                            + model.greysnag[i,j,t]

# Constraint 7: Own cell dispersal weight. 
def own_cell_dispersal(model):
    return

# Constraint 8:
def out_cell_dispersal(model):
    return

# Constraint 9:
def adding_up(model):
    return

# Constraint 10: 
def postmigration_infection(model):
    return

# Constraint 11:
def posttreatment_infection(model):
    return



SyntaxError: invalid syntax (<ipython-input-13-25c1519ed767>, line 3)

In [ ]:
################################################################################################################
# Random Parameter Generation for Long Range Dispersal 
# !!! NEED TO ASK IN STACK OVERFLOW !!!
#def Randlrd_init(model,i,j):
#    mu = []
#    for i in range(1,3):
#        for j in range(1,3):
#            new_rand = random.randrange(0, 0.1, 1)
#            mu.append(new_rand)
#    return mu
    
#model.v = Param(model.Iset, model.Js, model.Ts,     # Random Long Range Dispersal Parameter
 #                   initialize=Randlrd_init, default=0)     
################################################################################################################